<center> <font size="7"> Flight logs dashboard </font> </center>

___
    Pierre BAGNARA
    pierre.bagnara47@gmail.com
    15 Novembre 2022
___
Notebook associé au projet Python ```TP_trajectoire_circulaire```. <br> 
Les cellules de code Python 3 sont exécutables par le raccourci-clavier ```shift+Entrée``` une fois la cellule sélectionnée.
___
Ce Notebook interactif permet de lire les fichiers de logs ```.csv``` dans lesquels les données du dernier vol (réel ou en simulation) sont sauvegardées. Une série de graphiques est restitués après l'exécution de l'ensemble des cellules du Notebook (onglet ```Run``` -> ```Run all cells```).
___

In [2]:
%reset -f

In [3]:
import csv
import ipywidgets
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory
from jupyter_dash import JupyterDash
from dash import dcc, html, Dash

In [4]:
def extract_csv_data(filename):
    file = open(filename, 'r')
    reader = csv.DictReader(file)
    data_list = list(reader)
    body_names = set(data_list_element['Crazyflie name'] for data_list_element in data_list)
    variable_names = [key for key in data_list[0].keys() if key != 'QTM packet timestamp (s)' and key != 'Crazyflie name']
    timestamps = sorted(set(float(data_list_element['QTM packet timestamp (s)']) for data_list_element in data_list))
    data = {timestamp: {body_name: {variable_name: None for variable_name in variable_names} for body_name in body_names} for timestamp in timestamps}
    for i in range(len(data_list)):
        for key in data_list[0].keys():
            if key != 'Crazyflie name' and key != 'QTM packet timestamp (s)':
                if data_list[i][key] == 'None':
                    data[float(data_list[i]['QTM packet timestamp (s)'])][data_list[i]['Crazyflie name']][key] = None
                else:
                    data[float(data_list[i]['QTM packet timestamp (s)'])][data_list[i]['Crazyflie name']][key] = float(data_list[i][key])
    return body_names, data

In [5]:
def initial_plot(data, body_names, flight_zone_boundaries, colors_association, fig):
    figure_children = [children for children in fig.children]
    
    # Plot x, y, and z vectors
    figure_children[2].add_trace(go.Scatter3d(x=[0, 1], y=[0, 0], z=[0, 0], line=dict(color='red', width=20),
                                              name='X axis', legendgroup='0', mode='lines'))
    figure_children[2].add_trace(go.Scatter3d(x=[0, 0], y=[0, 1], z=[0, 0], line=dict(color='green', width=20),
                                              name='Y axis', legendgroup='0', mode='lines'))
    figure_children[2].add_trace(go.Scatter3d(x=[0, 0], y=[0, 0], z=[0, 1], line=dict(color='blue', width=20),
                                              name='Z axis', legendgroup='0', mode='lines'))
    
    timestamps = data.keys()
    t = [timestamp for timestamp in data.keys()]
    time = t[0]
    size = 10
    for name in body_names:
        color = colors_association[name]
        x = [data[timestamp][name]['QTM_x (m)'] for timestamp in timestamps]
        y = [data[timestamp][name]['QTM_y (m)'] for timestamp in timestamps]
        z = [data[timestamp][name]['QTM_z (m)'] for timestamp in timestamps]
        xg = [data[timestamp][name]['x_g (m)'] for timestamp in timestamps]
        yg = [data[timestamp][name]['y_g (m)'] for timestamp in timestamps]
        phi = [data[timestamp][name]['roll_c (°)'] for timestamp in timestamps]
        theta = [data[timestamp][name]['pitch_c (°)'] for timestamp in timestamps]
        psi_dot = [data[timestamp][name]['yaw_rate_c (°/s)'] for timestamp in timestamps]
        yaw = [data[timestamp][name]['cf_yaw (°)'] for timestamp in timestamps]
        yaw_g = [data[timestamp][name]['yaw_g (°)'] for timestamp in timestamps]
        
        # 3D graph : Main line
        figure_children[2].add_trace(go.Scatter3d(x=x, y=y, z=z,
                                                  line=dict(color=color),
                                                  name=name + ' trajectory',
                                                  legendgroup=name,
                                                  mode='lines',
                                                  connectgaps=False))
        # 3D graph : Position
        figure_children[2].add_trace(go.Scatter3d(x=[data[time][name]['QTM_x (m)']],
                                                  y=[data[time][name]['QTM_y (m)']],
                                                  z=[data[time][name]['QTM_z (m)']],
                                                  line=dict(color=color),
                                                  name=name + ' position',
                                                  legendgroup=name,
                                                  showlegend=False,
                                                  mode='markers',
                                                  marker=dict(size=4)))
        # 3D graph : Position goal
        figure_children[2].add_trace(go.Scatter3d(x=[data[time][name]['x_g (m)']],
                                                  y=[data[time][name]['y_g (m)']],
                                                  z=[data[time][name]['z_g (m)']],
                                                  line=dict(color=color, dash='dash'),
                                                  name=name + ' targeted position',
                                                  legendgroup=name,
                                                  mode='markers',
                                                  marker=dict(size=2)))
        
        # X graph
        figure_children[3].add_trace(go.Scatter(x=t, y=x,
                                                line=dict(color=color),
                                                name=name + ' X vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[3].add_trace(go.Scatter(x=[t[0]], y=[x[0]],
                                                line=dict(color=color),
                                                name=name + ' X',
                                                legendgroup=name,
                                                showlegend=False,
                                                mode='markers',
                                                marker=dict(size=size)))
        figure_children[3].add_trace(go.Scatter(x=t, y=xg,
                                                line=dict(color=color, dash='dash'),
                                                name=name + ' targeted X vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[3].update_xaxes(title='Time (s)')
        figure_children[3].update_yaxes(title='X (m)')
        
        # Y graph
        figure_children[4].add_trace(go.Scatter(x=t, y=y,
                                                line=dict(color=color),
                                                name=name + ' Y vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[4].add_trace(go.Scatter(x=[t[0]], y=[y[0]],
                                                line=dict(color=color),
                                                name=name + ' Y',
                                                legendgroup=name,
                                                showlegend=False,
                                                mode='markers',
                                                marker=dict(size=size)))
        figure_children[4].add_trace(go.Scatter(x=t, y=yg,
                                                line=dict(color=color, dash='dash'),
                                                name=name + ' targeted Y vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[4].update_xaxes(title='Time (s)')
        figure_children[4].update_yaxes(title='Y (m)')
        
        # Roll command graph
        figure_children[5].add_trace(go.Scatter(x=t, y=phi,
                                                line=dict(color=color),
                                                name=name + ' Roll command vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[5].add_trace(go.Scatter(x=[t[0]], y=[phi[0]],
                                                line=dict(color=color),
                                                name=name + ' Roll command',
                                                legendgroup=name,
                                                showlegend=False,
                                                mode='markers',
                                                marker=dict(size=size)))
        figure_children[5].update_xaxes(title='Time (s)')
        figure_children[5].update_yaxes(title='Roll command (°)')
        
        # Pitch command graph
        figure_children[6].add_trace(go.Scatter(x=t, y=theta,
                                                line=dict(color=color),
                                                name=name + ' Pitch command vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[6].add_trace(go.Scatter(x=[t[0]], y=[theta[0]],
                                                line=dict(color=color),
                                                name=name + ' Pitch command',
                                                legendgroup=name,
                                                showlegend=False,
                                                mode='markers',
                                                marker=dict(size=size)))
        figure_children[6].update_xaxes(title='Time (s)')
        figure_children[6].update_yaxes(title='Pitch command (°)')
        
        # Yaw graph
        figure_children[7].add_trace(go.Scatter(x=t, y=yaw,
                                                line=dict(color=color),
                                                name=name + ' Yaw vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[7].add_trace(go.Scatter(x=[t[0]], y=[yaw[0]],
                                                line=dict(color=color),
                                                name=name + ' Yaw',
                                                legendgroup=name,
                                                showlegend=False,
                                                mode='markers',
                                                marker=dict(size=size)))
        figure_children[7].add_trace(go.Scatter(x=t, y=yaw_g,
                                                line=dict(color=color, dash='dash'),
                                                name=name + ' targeted Yaw vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[7].update_xaxes(title='Time (s)')
        figure_children[7].update_yaxes(title='Yaw (°)')
        
        # Yaw rate command graph
        figure_children[8].add_trace(go.Scatter(x=t, y=psi_dot,
                                                line=dict(color=color),
                                                name=name + ' Yaw rate command vs time',
                                                legendgroup=name,
                                                mode='lines',
                                                connectgaps=False))
        figure_children[8].add_trace(go.Scatter(x=[t[0]], y=[yaw[0]],
                                                line=dict(color=color),
                                                name=name + ' Yaw rate command',
                                                legendgroup=name,
                                                showlegend=False,
                                                mode='markers',
                                                marker=dict(size=size)))
        figure_children[8].update_xaxes(title='Time (s)')
        figure_children[8].update_yaxes(title='Yaw rate (°/s)')
        

    figure_children[2].update_scenes(xaxis=dict(range=[flight_zone_boundaries[0][0], flight_zone_boundaries[0][1]],
                                                title='X (m)'),
                                     yaxis=dict(range=[flight_zone_boundaries[1][0], flight_zone_boundaries[1][1]],
                                                title='Y (m)'),
                                     zaxis=dict(range=[flight_zone_boundaries[2][0], flight_zone_boundaries[2][1]],
                                                title='Z (m)'),
                                     aspectmode='manual',
                                     aspectratio=dict(x=1,
                                                      y=abs((flight_zone_boundaries[1][1] - flight_zone_boundaries[1][0]) /
                                                            (flight_zone_boundaries[0][1] - flight_zone_boundaries[0][0])),
                                                      z=abs((flight_zone_boundaries[2][1] - flight_zone_boundaries[2][0]) /
                                                            (flight_zone_boundaries[0][1] - flight_zone_boundaries[0][0])))
                                    )
    return fig

In [6]:
def update_plots_data(data, body_names, timestamp_range, time):
    timestamps = data.keys()
    plot_data = {'X axis': dict(x=[0, 1], y=[0, 0], z=[0, 0]),
                 'Y axis': dict(x=[0, 0], y=[0, 1], z=[0, 0]),
                 'Z axis': dict(x=[0, 0], y=[0, 0], z=[0, 1])}
    t = [timestamp for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
    for name in body_names:
        x = [data[timestamp][name]['QTM_x (m)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        y = [data[timestamp][name]['QTM_y (m)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        z = [data[timestamp][name]['QTM_z (m)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        xg = [data[timestamp][name]['x_g (m)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        yg = [data[timestamp][name]['y_g (m)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        phi = [data[timestamp][name]['roll_c (°)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        theta = [data[timestamp][name]['pitch_c (°)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        psi_dot = [data[timestamp][name]['yaw_rate_c (°/s)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        yaw = [data[timestamp][name]['cf_yaw (°)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        yaw_g = [data[timestamp][name]['yaw_g (°)'] for timestamp in timestamps if timestamp_range[0] <= timestamp <= timestamp_range[1]]
        
        plot_data[name + ' trajectory'] = dict(x=x, y=y, z=z)
        plot_data[name + ' position'] = dict(x=[data[time][name]['QTM_x (m)']],
                                             y=[data[time][name]['QTM_y (m)']],
                                             z=[data[time][name]['QTM_z (m)']])
        plot_data[name + ' targeted position'] = dict(x=[data[time][name]['x_g (m)']],
                                                      y=[data[time][name]['y_g (m)']],
                                                      z=[data[time][name]['z_g (m)']])
        plot_data[name + ' X vs time'] = dict(x=t, y=x)
        plot_data[name + ' X'] = dict(x=[time], y=[data[time][name]['QTM_x (m)']])
        plot_data[name + ' targeted X vs time'] = dict(x=t, y=xg)
        plot_data[name + ' Y vs time'] = dict(x=t, y=y)
        plot_data[name + ' Y'] = dict(x=[time], y=[data[time][name]['QTM_y (m)']])
        plot_data[name + ' targeted Y vs time'] = dict(x=t, y=yg)
        plot_data[name + ' Roll command vs time'] = dict(x=t, y=phi)
        plot_data[name + ' Roll command'] = dict(x=[time], y=[data[time][name]['roll_c (°)']])
        plot_data[name + ' Pitch command vs time'] = dict(x=t, y=theta)
        plot_data[name + ' Pitch command'] = dict(x=[time], y=[data[time][name]['pitch_c (°)']])
        plot_data[name + ' Yaw vs time'] = dict(x=t, y=yaw)
        plot_data[name + ' Yaw'] = dict(x=[time], y=[data[time][name]['cf_yaw (°)']])
        plot_data[name + ' targeted Yaw vs time'] = dict(x=t, y=yaw_g)
        plot_data[name + ' Yaw rate command vs time'] = dict(x=t, y=psi_dot)
        plot_data[name + ' Yaw rate command'] = dict(x=[time], y=[data[time][name]['yaw_rate_c (°/s)']])
    return plot_data

In [7]:
def update_plots(fig, plot_data):
    children = [ch for ch in fig.children]
    for i in range(len(children)):
        if i >= 2:
            with children[i].batch_update():
                for plot in children[i].data:
                    plot.x = plot_data[plot.name]['x']
                    plot.y = plot_data[plot.name]['y']
                    try:
                        plot.z = plot_data[plot.name]['z']
                    except KeyError:
                        pass
    return fig

In [8]:
def time_range_changed(change):
    global figure
    global time_slider
    global data
    global bodies_names
    
    new_range = change['new']
    time_slider.min = new_range[0]
    time_slider.max = new_range[1]
    
    timestamps = [timestamp for timestamp in data.keys()]
    timestamp_range = [timestamps[new_range[0]], timestamps[new_range[1]]]
    time = timestamps[time_slider.value]
    plot_data = update_plots_data(data, bodies_names, timestamp_range, time)
    figure = update_plots(figure, plot_data)

In [9]:
def time_changed(change):
    global figure
    global range_slider
    global data
    global bodies_names
    
    new_time = change['new']
    timestamps = [timestamp for timestamp in data.keys()]
    timestamp_range = [timestamps[range_slider.value[0]], timestamps[range_slider.value[1]]]
    time = timestamps[new_time]
    plot_data = update_plots_data(data, bodies_names, timestamp_range, time)
    figure = update_plots(figure, plot_data)

In [10]:
def display_dashboard(filename):
    global figure
    global time_slider
    global range_slider
    global data
    global bodies_names

    flight_area_limits = [[-2.5, 2.5], [-2.5, 2.5], [-0.1, 2]]
    cf_colors = dict(Crazyflie='cyan', Cible='red')

    bodies_names, data = extract_csv_data(filename)
    timestamps = [timestamp for timestamp in data.keys()]
    range_slider = ipywidgets.IntRangeSlider(value=[0, len(timestamps) - 1], min=0, max=len(timestamps) - 1, step=1, 
                                             description='Time range', readout=False, layout=ipywidgets.Layout(width='1000px'))
    time_slider = ipywidgets.IntSlider(value=0, min=0, max=len(timestamps) - 1, step=1, 
                                       description= 'Time', readout=False, layout=ipywidgets.Layout(width='1000px'))
    height = 350
    figure = ipywidgets.VBox([range_slider,
                              time_slider,
                              go.FigureWidget(data=[], layout=dict(title_text='3D trajectory',
                                                                                 template='plotly_dark',
                                                                                 legend_tracegroupgap=10,
                                                                                 height=2*height)),
                              go.FigureWidget(data=[], layout=dict(title_text='X vs time',
                                                                               template='plotly_dark',
                                                                               legend_tracegroupgap=10,
                                                                               height=height)),
                              go.FigureWidget(data=[], layout=dict(title_text='Y vs time',
                                                                               template='plotly_dark',
                                                                               legend_tracegroupgap=10,
                                                                               height=height)),
                              go.FigureWidget(data=[], layout=dict(title_text='Roll command vs time',
                                                                               template='plotly_dark',
                                                                               legend_tracegroupgap=10,
                                                                               height=height)),
                              go.FigureWidget(data=[], layout=dict(title_text='Pitch command vs time',
                                                                               template='plotly_dark',
                                                                               legend_tracegroupgap=10,
                                                                               height=height)),
                              go.FigureWidget(data=[], layout=dict(title_text='Yaw vs time',
                                                                               template='plotly_dark',
                                                                               legend_tracegroupgap=10,
                                                                               height=height)),
                              go.FigureWidget(data=[], layout=dict(title_text='Yaw rate command vs time',
                                                                               template='plotly_dark',
                                                                               legend_tracegroupgap=10,
                                                                               height=height))])
    range_slider.observe(time_range_changed, names='value')
    time_slider.observe(time_changed, names='value')
    figure = initial_plot(data, bodies_names, flight_area_limits, cf_colors, figure)
    display(figure)

In [11]:
global figure
global time_slider
global range_slider
global data
global bodies_names
# display_dashboard('sim_logs.csv')
display_dashboard('logs.csv')